In [2]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import AdaBoostClassifier

In [3]:
df_raw = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test_raw = pd.read_csv("../data/test.csv").drop("id",axis=1)

In [4]:
# Récupération des labels
labels = df_raw["target"].values
y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)
# Récupérations des features sous forme de ndarray
df_raw = df_raw.drop("target", axis=1)
X = df_raw.values
# X_test = df_test_raw.values
print('X shape : '+str(X.shape))
print('y shape : '+str(y.shape))

X shape : (61878, 93)
y shape : (61878,)


In [5]:
def get_bin_model(X, y, c):
    X_with_c = X[np.where(y == (c - 1))]
    X_without_c = X[np.where(y != (c-1))]
    np.random.shuffle(X_without_c)
    X_without_c = X_without_c[:X_with_c.shape[0]]
    print('X shapes : ')
    print(X_with_c.shape)
    print(X_without_c.shape)
    X = np.concatenate((X_with_c, X_without_c), axis=0)
    print(X.shape)
    print('y shape : ')
    y_1 = np.array([1 for i in range(X_with_c.shape[0])])
    y_2 = np.array([0 for i in range(X_without_c.shape[0])])
    y = np.concatenate((y_1, y_2))
    print(y.shape)
    X_norm = Normalizer().fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2,
    random_state=42)
    
    clf_tmp_1 = ExtraTreesClassifier(n_estimators=20, random_state=0, n_jobs=-1)
    clf_tmp_2 = CalibratedClassifierCV(clf_tmp_1, cv=5)

    clf = AdaBoostClassifier(clf_tmp_2,
    n_estimators=500,
    learning_rate=1.0)
    %time clf.fit(X_train, y_train)

    pred_for_acc = clf.predict(X_test)

    #print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
    print(classification_report(y_test, pred_for_acc, target_names=['class_'+str(c-1), 'other']))
    
    return clf
    

In [6]:
def encode_proba(clf, c,X_test):
    preds = []
    pb = clf.predict_proba(X_test)
    for p in pb:
        pred = [0.0 for i in range(9)]
        pred[c-1] = p[0]
        preds.append(pred)
    
    return np.array(preds)

In [7]:
def mix_proba(pred_1, pred_2, coef_1, coef_2):
    a1 = (pred_1 * coef_1)
    #print(a1.shape)
    a2 = (pred_2 * coef_2)
    #print(a2.shape)
    s = (pred_1 * coef_1) + (pred_2 * coef_2)
    #print(s.shape)
    return s

In [63]:
def mix_proba_lin(pred_1, pred_2, coef):
    return ((coef * pred_1) + (1 - coef) * pred_2)

In [8]:
test = get_bin_model(X,y,4)

X shapes : 
(2691, 93)
(2691, 93)
(5382, 93)
y shape : 
(5382,)
CPU times: user 1.59 s, sys: 144 ms, total: 1.74 s
Wall time: 2.23 s
             precision    recall  f1-score   support

    class_3       0.89      0.84      0.86       510
      other       0.86      0.91      0.89       567

avg / total       0.88      0.88      0.88      1077



In [9]:
X_norm = Normalizer().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']


In [10]:
p = encode_proba(test, 4, X_test)

In [11]:
print(p.shape)

(12376, 9)


In [12]:
clf_tmp = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
clf.fit(X_train, y_train)
print("Fit terminé")
pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

Fit terminé
0.47
             precision    recall  f1-score   support

    class_1       0.78      0.58      0.66       399
    class_2       0.77      0.84      0.80      3178
    class_3       0.62      0.59      0.61      1561
    class_4       0.81      0.55      0.65       538
    class_5       0.97      0.98      0.97       565
    class_6       0.94      0.95      0.94      2884
    class_7       0.74      0.69      0.72       552
    class_8       0.90      0.92      0.91      1674
    class_9       0.87      0.89      0.88      1025

avg / total       0.83      0.83      0.82     12376



In [68]:
_, knn_pred = pickle.load(open('../data/knn300.p', 'rb'))

In [69]:
print(knn_pred.shape)
print(pred.shape)

(12376, 9)
(12376, 9)


In [59]:
coef_4 = [0.0 for i in range(9)]
coef_4[4-1] = 0.9
coef = [1-x for x in coef_4]
coef_4 = np.array(coef_4)
coef = np.array(coef)

In [60]:
mixed = mix_proba(p, pred, coef_4, coef)

In [61]:
mixed == pred

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [79]:
svm = pickle.load(open('../data/svm.p', 'rb'))
print(svm.shape)
ext = pickle.load(open('../data/extdc.p', 'rb'))
print(ext.shape)

(144368, 9)
(144368, 9)


In [80]:
for c in range(100):
    cc = c / 100.0
    m = mix_proba_lin(ext,svm, cc)
    print("%.2f" % log_loss(y_test, m, eps=1e-15))

ValueError: Found input variables with inconsistent numbers of samples: [144368, 12376]

In [56]:
print(p[2])
print(coef)
c = coef.reshape(9,1)
pp = np.dot(p, c)
print((pp[2]))


[ 0.          0.          0.          0.98169192  0.          0.          0.
  0.          0.        ]
[[ 1.   1.   1.   1.   0.3  1.   1.   1.   1. ]]
[ 0.98169192]


# Brouillon

In [27]:
print(X_4.shape)
X_r = X[np.where(y != 3)]
np.random.shuffle(X_r)
X_not_4 = X_r[:X_4.shape[0]]
print(X_not_4.shape)

(2691, 93)
(2691, 93)


In [29]:
X = np.concatenate((X_4, X_not_4), axis=0)
y_1 = np.array([1 for i in range(X_4.shape[0])])
y_2 = np.array([0 for i in range(X_4.shape[0])])
y = np.concatenate((y_1, y_2))
print(X.shape)
print(y.shape)

(5382, 93)
(5382,)


In [31]:
X_norm = Normalizer().fit_transform(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [36]:
clf = SVC(kernel='linear', C=1.0)
%time clf.fit(X_train, y_train)
#%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
#print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 1.24 s, sys: 20.6 ms, total: 1.26 s
Wall time: 1.3 s
Accuracy : 0.81
             precision    recall  f1-score   support

    class_1       0.88      0.69      0.77       510
    class_2       0.77      0.91      0.83       567

avg / total       0.82      0.81      0.80      1077



/Users/nour/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 9
  .format(len(labels), len(target_names))


In [54]:
#clf = RandomForestClassifier(n_estimators=30, n_jobs=-1)
#clf_tmp = ExtraTreesClassifier(n_estimators=30, random_state=0, n_jobs=-1)
#clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
clf_tmp_1 = ExtraTreesClassifier(n_estimators=20, random_state=0, n_jobs=-1)
clf_tmp_2 = CalibratedClassifierCV(clf_tmp_1, cv=5)

clf = AdaBoostClassifier(clf_tmp_2,
    n_estimators=500,
    learning_rate=1.0)
%time clf.fit(X_train, y_train)
#%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
#print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 1.19 s, sys: 89.2 ms, total: 1.28 s
Wall time: 1.76 s
Accuracy : 0.87
             precision    recall  f1-score   support

    class_1       0.89      0.83      0.86       510
    class_2       0.86      0.91      0.88       567

avg / total       0.87      0.87      0.87      1077



/Users/nour/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 9
  .format(len(labels), len(target_names))
